In [ ]:
import numpy as np
import lmdb
import pickle

In [ ]:
# total_imgs = !find {image_folder_path} -type f -iname "*.png" | wc -l

In [ ]:
env = lmdb.open('/home/jupyter/datasets/lsun-car/car/', readonly=True)
with env.begin() as txn:
    cursor = txn.cursor()
    count = 0
    for key, value in cursor:
        print(key)

        # Remember it's a CIFAR_Image object that is loaded
        cifar_image = pickle.loads(value)
        # Retrieve the relevant bits
        print(cifar_image)
        if count > 15:
            break
        count += 1

In [ ]:
from __future__ import print_function
import argparse
import cv2
import lmdb
import numpy
from matplotlib import pyplot as plt
import os
from os.path import exists, join

def view(db_path, limit):
    print('Viewing', db_path)
    print('Press ESC to exist or SPACE to advance.')
    window_name = 'LSUN'
    env = lmdb.open(db_path, map_size=1099511627776,
                    max_readers=100, readonly=True)
    with env.begin(write=False) as txn:
        cursor = txn.cursor()
        for i, (key, val) in enumerate(cursor):
            print('Current key:', key)
            img = cv2.imdecode(
                numpy.frombuffer(val, dtype=numpy.uint8), 1)
            plt.imshow(img)
            plt.title('my picture')
            plt.show()
            if i == limit: break


def export_images(db_path, out_dir, flat=False, limit=-1):
    print('Exporting', db_path, 'to', out_dir)
    env = lmdb.open(db_path, map_size=1099511627776,
                    max_readers=100, readonly=True)
    count = 0
    with env.begin(write=False) as txn:
        cursor = txn.cursor()
        for key, val in cursor:
            if not flat:
                print(key[:6].decode("ascii"))
                image_out_dir = join(out_dir, '/'.join(key[:6].decode("ascii")))
            else:
                image_out_dir = out_dir
            if not exists(image_out_dir):
                os.makedirs(image_out_dir)
            image_out_path = join(image_out_dir, key.decode("ascii") + '.webp')
            with open(image_out_path, 'w') as fp:
                fp.write(val)
            count += 1
            if count == limit:
                break
            if count % 1000 == 0:
                print('Finished', count, 'images')                
                

lmdb_path = '/home/jupyter/datasets/lsun-car/car/'
out_path  = '/home/jupyter/lsun_imgs/lsun_car'
view(lmdb_path, 2)
export_images(lmdb_path, out_path, limit=20)

In [ ]:
join("fdsa", "fdja;l")

In [ ]:
import os
from PIL import Image

path = '/home/jupyter/lsun_imgs/lsun_church/'
dirs = os.listdir(path)

# print(dirs)

for i, img_fn in enumerate(dirs):
  if i > 15:
    break
  im = Image.open(path + img_fn)
  display(im)

# path = '/content/results/stylegan2_cats/'
# dirs = os.listdir(path)

# print(dirs)

# for i, img_fn in enumerate(dirs):
#   if i > 15:
#     break
#   im = Image.open(path + img_fn)
#   display(im)

# path = '/content/results/stylegan2_church/'
# dirs = os.listdir(path)

# print(dirs)

# for i, img_fn in enumerate(dirs):
#   if i > 15:
#     break
#   im = Image.open(path + img_fn)
#   display(im)

In [ ]:
import os
from PIL import Image

path = '/home/jupyter/image_folder/'
dirs = os.listdir(path)

print(dirs)

In [ ]:
%run DeepFake_Dataloader.py
%run utils.py

In [ ]:
path = '/home/jupyter/lsun_imgs/lsun_church/'
dirs = os.listdir(path)

# print(dirs)

for i, img_fn in enumerate(dirs):
    if i > 15:
        break
    img = pil_grey_loader(path + img_fn)
    display(img)
    ms_img = np_magnitude_spectrum(img)
    rad_p = np_radial_profile(ms_img, center=(ms_img.shape[0]/2, ms_img.shape[1]/2))
    print(ms_img.shape)
    print(rad_p.shape)




In [ ]:
fake_dir = '/home/jupyter/image_folder/stylegan1_bedrooms_30k'
real_dir = '/home/jupyter/image_folder/lsun_bedrooms'

import os
from PIL import Image
import random

fake_imgs = os.listdir(fake_dir)
real_imgs = os.listdir(real_dir)

print(random.choice(fake_imgs))

def create_collage(width, height, path, listofimages, fn):
    cols = 9
    rows = 6
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    for p in listofimages:
        im = Image.open(path + '/' + p)
        im.thumbnail(size)
        ims.append(im)
    i = 0
    x = 0
    y = 0
    for col in range(cols):
        for row in range(rows):
            print(i, x, y)
            new_im.paste(ims[i], (x, y))
            i += 1
            y += thumbnail_height
        x += thumbnail_width
        y = 0
    
    display(new_im)
    fn = fn + '.jpg'
    new_im.save(fn)
    return new_im, fn, new_im.size

fake_list = []
real_list = []
for i in range(108):
    fake_list.append(random.choice(fake_imgs))
    real_list.append(random.choice(real_imgs))

fake_ims, fake_fn, fake_sz = create_collage(450, 300, fake_dir, fake_list, 'bedroom_fake')
real_ims, real_fn, real_sz = create_collage(450, 300, real_dir, real_list, 'bedroom_real')

new_im = Image.new('RGB', size=(fake_sz[0] + real_sz[0], 300), color=(255,255,255,0))
new_im.paste(fake_ims, (0, 0))
new_im.paste(real_ims, (fake_sz[0] + 5, 0))
collage_fn = 'churches_collage.jpg'
display(new_im)
new_im.save(collage_fn)

In [ ]:
# fake_dir = '/home/jupyter/image_folder/stylegan1_bedrooms_30k'
# real_dir = '/home/jupyter/image_folder/lsun_bedrooms'

import os
from PIL import Image
import random

import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob


origin_dataset = {
    'lsun_bedrooms': 0, 
    'lsun_cars': 1, 
    'lsun_cats': 2, 
    'lsun_churches': 3, 
    'stylegan1_bedrooms': 4, 
    'stylegan2_cars': 5, 
    'stylegan2_cats': 6, 
    'stylegan2_churches': 7, 
    0: 'lsun_bedrooms', 
    1: 'lsun_cars', 
    2: 'lsun_cats', 
    3: 'lsun_churches', 
    4: 'stylegan1_bedrooms', 
    5: 'stylegan2_cars', 
    6: 'stylegan2_cats', 
    7: 'stylegan2_churches'
}

origin_dataset = {
    'lsun_cars': 0, 
    'stylegan2_cars': 1, 
    0: 'lsun_cars', 1: 
    'stylegan2_cars'
}


y            = []
error        = []
lbls         = []
in_vector_sz = 182

path = '/home/jupyter/CSE253_FinalProject/LSUN_BCCC_256.hdf5'
dt   = {}
plts = {}

with h5py.File(path, 'r') as f:
    dset = f['fft_data']
    orgn = f['orgn_data']
    total_imgs = len(dset)
    x = np.arange(0, in_vector_sz)
    
    for z in range(total_imgs):
        dataset_name = origin_dataset[orgn[z]]
        if dataset_name not in dt:
            dt[dataset_name] = [z]
        else:
            dt[dataset_name].append(z)
    for key, val in dt.items():
        arr = np.array(dset[val])
        y.append(np.mean(arr, axis=0))
        error.append(np.std(arr, axis=0))
        lbls.append(key)
        plts[key] = (np.mean(arr, axis=0), np.std(arr, axis=0))
    
        

fig, ax = plt.subplots(figsize=(15, 9))
for i in range(len(y)):
    color = list(mcolors.TABLEAU_COLORS.keys())[i]
    ax.plot(x, y[i], alpha=0.5, color=mcolors.TABLEAU_COLORS[color], label=lbls[i], linewidth =2.0)
    ax.fill_between(x, y[i] - error[i], y[i] + error[i], color=mcolors.TABLEAU_COLORS[color], alpha=0.2)
    

# ax.plot(x, y[1], alpha=0.5, color='blue', label='100K Faces (fake)', linewidth = 2.0)
# ax.fill_between(x, y[1] - error[1], y[1] + error[1], color='blue', alpha=0.2)

# ax.plot(x, y[2], alpha=0.5, color='green', label='Flickr-Faces-HQ (real)', linewidth = 2.0)
# ax.fill_between(x, y[2] - error[2], y[2] + error[2], color='green', alpha=0.2)

# ax.plot(x, y[3], alpha=0.5, color='brown', label='CelebA-HQ (real)', linewidth = 2.0)
# ax.fill_between(x, y[3] - error[3], y[3] + error[3], color='brown', alpha=0.2)

ax.set_title('Satistics 1D Power Spectrum',size=20)
plt.xlabel('Spatial Frequency', fontsize=20)
plt.ylabel('Power Spectrum', fontsize=20)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
ax.legend(loc='best', prop={'size': 20})
plt.savefig('mean_plt_bccc.jpg', dpi=300)
plt.show()

In [ ]:
fake_sz = (1, 3)
print(fake_sz)
fake_sz = list(fake_sz)
print(fake_sz)

In [20]:
import numpy as np
import os

string_ex = "Hello"
a = np.string_(string_ex)
b = np.frombuffer(a, dtype=np.uint8)
print(type(b))
c = np.char.decode(np.ndarray.tostring(b))
print(c)
print(b)
print(a.shape)
print(a)
print(str(np.char.decode(a)))
path_ex = '/home/jupyter/image_folder_BCCC_256/lsun_cats/img00001834.png'
print(len(path_ex))
split_path = os.path.split(path_ex)
print(split_path)
split_path = os.path.split(split_path[0])
print(split_path)

<class 'numpy.ndarray'>
Hello
[ 72 101 108 108 111]
()
b'Hello'
Hello
61
('/home/jupyter/image_folder_BCCC_256/lsun_cats', 'img00001834.png')
('/home/jupyter/image_folder_BCCC_256', 'lsun_cats')


In [36]:
import torch
a = torch.Tensor([0, 1, 0, 1, 0])
c = torch.Tensor([1, 0.5, 2, 0, 1.8])
b = torch.Tensor([1, 1, 1, 1, 1])
d = torch.Tensor([0, 0, 3, 1, 1])
e = (a, c)
f = (b, d)
g = a[:] * b[:]
print(g)
h = c[:] * d[:]
print(h)
i = e[0][:] * f[0][:]
print(i)

tensor([0., 1., 0., 1., 0.])
tensor([0.0000, 0.0000, 6.0000, 0.0000, 1.8000])
tensor([0., 1., 0., 1., 0.])
